
# Table of Contents







## 5 SAT問題の例



Google Colabの場合は最初に以下を実行してください．



In [1]:
! if [ ! -e "/usr/bin/java" ]; then apt update && apt install openjdk-11-jre-headless; fi
! git clone https://github.com/tamura70/cspsat-jupyter.git
%cd cspsat-jupyter

### 5.1 嘘つきパズル



例題「嘘つきパズル」のCNF式



In [1]:
from cspsat import *
(A, B) = (Bool("A"), Bool("B"))
cnf = [ [~A, ~B], [A, B], [A, ~B] ]

solveSAT関数による最初の解の表示



In [1]:
solveSAT(cnf)

solveSAT関数によるすべての解の表示



In [1]:
solveSAT(cnf, num=0)

例題「嘘つきパズル」でA=1, B=0以外の解を探す



In [1]:
cnf = [ [~A, ~B], [A, B], [A, ~B], [~A, B] ]
solveSAT(cnf)

例題「嘘つきパズル」のCNF式 (yield命令を使用)



In [1]:
def liarPuzzle():
    yield [~A, ~B]
    yield [A, B]
    yield [A, ~B]

In [1]:
for clause in liarPuzzle():
    print(clause)

In [1]:
list(liarPuzzle())

In [1]:
solveSAT(liarPuzzle())

In [1]:
def liarPuzzle2():
    for clause in liarPuzzle():
        yield clause
    yield [~A, B]

yield from命令を用いたCNF式の定義の例



In [1]:
def liarPuzzle2():
    yield from liarPuzzle()
    yield [~A, B]

In [1]:
list(liarPuzzle2())

itertools.chain関数の使用例



In [1]:
import itertools
solveSAT(itertools.chain(liarPuzzle(), [ [~A,B] ]))

リストのアンパックの使用例



In [1]:
[*liarPuzzle(), [~A, B]]

itertools.combinations関数の使用例



In [1]:
list(itertools.combinations([1,2,3,4], 2))

itertools.permutations関数の使用例



In [1]:
list(itertools.permutations([1,2,3,4], 2))

itertools.prodcut関数の使用例



In [1]:
list(itertools.product([1,2,3], ["a","b"]))

In [1]:
from itertools import chain, combinations, permutations, product


# Table of Contents







#### 練習問題 5.1.1: CNF式を求める



In [1]:
(A, B) = (Bool("A"), Bool("B"))
toCNF(["and", ["equ", A, ~B], ["equ", B, ["and", A, B]]])

#### 練習問題 5.1.2: 嘘つきパズルの例1



In [1]:
(A, B) = (Bool("A"), Bool("B"))
cnf = [ [~A, ~B], [~B], [A, B] ]
solveSAT(cnf, num=0)

### 5.2 班分け



例題「班分けパズル」のCNF式



In [1]:
from cspsat import *
def grouping():
    (A, B, C, D) = (Bool("A"), Bool("B"), Bool("C"), Bool("D"))
    yield from toCNF(["not", ["equ", A, B]])
    yield from toCNF(["not", ["equ", B, C]])
    yield from toCNF(["not", ["equ", C, D]])
    yield from toCNF(["not", ["equ", D, A]])

In [1]:
list(grouping())

すべての解をsolveSAT関数で探索



In [1]:
solveSAT(grouping(), num=0)

対称解を除去して探索



In [1]:
A = Bool("A")
solveSAT([*grouping(), [~A]], num=0)


# Table of Contents







### 5.3 ファン・デル・ヴェルデン数



長さnの2進列に，3つの0も3つの1も等間隔で現れない



In [1]:
from cspsat import *
def waerden3(n, x=Bool("x")):
    for d in range(1,n):
        for i in range(1,n+1):
            if i+2*d <= n:
                yield [ x(i), x(i+d), x(i+2*d) ]
                yield [ ~x(i), ~x(i+d), ~x(i+2*d) ]

In [1]:
list(waerden3(8))

長さ8のときの6通りの解を探索



In [1]:
solveSAT(waerden3(8), num=0)

長さ9だと解は存在しない



In [1]:
solveSAT(waerden3(9))

長さnの2進列に，k個の0もk個の1も等間隔で現れない



In [1]:
def waerden(k, n, x=Bool("x")):
    for d in range(1,n):
        for i in range(1,n+1):
            if i+(k-1)*d <= n:
                yield [ x(i+j*d) for j in range(k) ]
                yield [ ~x(i+j*d) for j in range(k) ]

In [1]:
def solveWaerden(k, n, num=1, x=Bool("x")):
    for sol in solutionsSAT(waerden(k, n), num=num):
        print([ sol[x(i)] for i in range(1,n+1) ])

k=4はn=34なら解が存在する



In [1]:
solveWaerden(4, 34)

k=4はn=35なら解が存在しない



In [1]:
solveWaerden(4, 35)


# Table of Contents







#### 練習問題 5.3.1: waerden3(8)の解の個数



In [1]:
solveSAT(waerden3(8), num=0)

#### 練習問題 5.3.2: W(2,5)=178の確認



In [1]:
solveSAT(waerden(5, 177), positiveOnly=True)
solveSAT(waerden(5, 178), positiveOnly=True)

### 5.4 1次元ライツアウト



lightsOut1d関数: すべてを消すためのCNF式



In [1]:
from cspsat import *
def lightsOut1d(lights, p=Bool("p")):
    n = len(lights)
    yield from toCNF(["and", ~p(0), ~p(n+1)])
    for i in range(1, n+1):
        if lights[i-1]:
            yield from toCNF(["xor", p(i-1), p(i), p(i+1)])
        else:
            yield from toCNF(["xor", ~p(i-1), p(i), p(i+1)])

10101から始めてすべてのライトを消す方法を探す



In [1]:
solveSAT(lightsOut1d([1,0,1,0,1]), num=0, positiveOnly=True)

10000だとすべてのライトを消すことはできない



In [1]:
solveSAT(lightsOut1d([1,0,0,0,0]), num=0, positiveOnly=True)


# Table of Contents







#### 練習問題 5.4.3: 2次元ライツアウト



In [1]:
from cspsat import *
def lightsOut2d(lights, p=Bool("p")):
    (m, n) = (len(lights), len(lights[0]))
    for i in [0,m+1]:
        yield from [ [~p(i,j)] for j in range(n+2) ]
    for j in [0,n+1]:
        yield from [ [~p(i,j)] for i in range(1,m+1) ]
    for i in range(1, m+1):
        for j in range(1, n+1):
            f = ["xor", p(i,j), p(i-1,j), p(i+1,j), p(i,j-1), p(i,j+1)]
            if lights[i-1][j-1]:
                yield from toCNF(f)
            else:
                yield from toCNF(["not", f])

In [1]:
n = 5
lights = [[1] * n] * n
solveSAT(lightsOut2d(lights), positiveOnly=True)

### 5.5 ちょうど1つ



eq1関数: exact-one制約



In [1]:
from cspsat import *
p = Bool("p")
list(eq1([ p(1), p(2), p(3) ]))

In [1]:
solveSAT(eq1([ p(1), p(2), p(3) ]), num=0)

例題「ちょうど1つ」のCNF式



In [1]:
def exactOne(n, x=Bool("x")):
    for i in range(1,n+1):
        yield from eq1([ x(i,j) for j in range(1,n+1) ])
    for j in range(1,n+1):
        yield from eq1([ x(i,j) for i in range(1,n+1) ])

In [1]:
solveSAT(exactOne(3), num=0, positiveOnly=True)


# Table of Contents







#### 練習問題 5.5.4: 鳩の巣原理の実行時間



In [1]:
def pigeonHolePrinciple(n, x=Bool("x")):
    for i in range(1,n+2):
        yield from ge1([ x(i,j) for j in range(1,n+1) ])
    for j in range(1,n+1):
        yield from le1([ x(i,j) for i in range(1,n+2) ])

In [1]:
solveSAT(pigeonHolePrinciple(8), stat=True)
solveSAT(pigeonHolePrinciple(9), stat=True)
solveSAT(pigeonHolePrinciple(10), stat=True)

### 5.6 推理パズル



例題「推理パズル1」の途中まで



In [1]:
from cspsat import *
def logicPuzzle1_0(x=Bool("x")):
    I = [ "論", "理", "学" ]
    J = [ "長男", "次男", "三男" ]
    for i in I:
        yield from eq1([ x(i,j) for j in J ])
    for j in J:
        yield from eq1([ x(i,j) for i in I ])

In [1]:
solveSAT(logicPuzzle1_0(), num=0, positiveOnly=True)

例題「推理パズル1」に条件の一部を追加



In [1]:
def logicPuzzle1_1(x=Bool("x")):
    yield from logicPuzzle1_0(x=x)
    yield [ ~x("学","長男") ]
    yield [ ~x("論","長男"), x("理","次男") ]
    yield [ ~x("論","次男"), x("理","三男") ]

solveSAT(logicPuzzle1_1(), num=0, positiveOnly=True)

例題「推理パズル1」のCNF式



In [1]:
def logicPuzzle1(x=Bool("x")):
    yield from logicPuzzle1_0(x=x)
    yield [ ~x("学","長男") ]
    yield [ ~x("論","長男"), x("理","次男") ]
    yield [ ~x("論","次男"), x("理","三男") ]
    yield [ ~x("論","三男") ]

solveSAT(logicPuzzle1(), num=0, positiveOnly=True)

例題「推理パズル1」のCNF式 (別の方法)



In [1]:
def logicPuzzle1(x=Bool("x")):
    yield from logicPuzzle1_0(x=x)
    yield [ ~x("学","長男") ]
    yield [ ~x("論","長男"), ~x("理","三男") ]
    yield [ ~x("論","次男"), ~x("理","長男") ]
    yield [ ~x("論","三男") ]

solveSAT(logicPuzzle1(), num=0, positiveOnly=True)

例題「推理パズル2」のCNF式



In [1]:
def logicPuzzle2(x=Bool("x")):
    I = [ "論", "理", "学" ]
    J = [ "袋", "箱", "缶" ]
    K = [ "赤", "緑", "青" ]
    for i in I:
        yield from eq1([ x(i,j,k) for j in J for k in K ])
    for j in J:
        yield from eq1([ x(i,j,k) for i in I for k in K ])
    for k in K:
        yield from eq1([ x(i,j,k) for i in I for j in J ])
    # 論は袋を買いました
    yield [ x("論","袋",k) for k in K ]
    # 理が買ったのは箱ではありません
    for k in K:
        yield [ ~x("理","箱",k) ]
    # 袋は青色です
    yield [ x(i,"袋","青") for i in I ]
    # 学が買ったものは赤色ではありません
    for j in J:
        yield [ ~x("学",j,"赤") ]

In [1]:
solveSAT(logicPuzzle2(), num=0, positiveOnly=True)


# Table of Contents







### 5.7 $n$クイーン



例題「nクイーン」のCNF式



In [1]:
from cspsat import *
def queens(n, x=Bool("x")):
    for i in range(n):
        yield from eq1([ x(i,j) for j in range(n) ])
    for j in range(n):
        yield from eq1([ x(i,j) for i in range(n) ])
    for a in range(0, 2*n-1):
        yield from le1([ x(i,a-i) for i in range(n) if a-i in range(n) ])
    for b in range(-n+1, n):
        yield from le1([ x(i,i-b) for i in range(n) if i-b in range(n) ])

In [1]:
solveSAT(queens(4), num=0, positiveOnly=True)

In [1]:
n = 4
x = Bool("x")
for sol in solutionsSAT(queens(n), num=0):
    for i in range(n):
        qs = [ "Q" if sol[x(i,j)] else "." for j in range(n) ]
        print(" ".join(qs))
    print()

8クイーンの全解



In [1]:
solveSAT(queens(8), num=0, positiveOnly=True)

100クイーンの最初の解



In [1]:
solveSAT(queens(100), positiveOnly=True, stat=True)


# Table of Contents







#### 練習問題 5.7.1: 点対称なnクイーン



In [1]:
def squeens(n, x=Bool("x")):
    yield from queens(n)
    for i in range(n):
        for j in range(n):
            yield [ ~x(i,j), x(n-1-i,n-1-j) ]

In [1]:
n = 8
x = Bool("x")
for sol in solutionsSAT(squeens(n)):
    for i in range(n):
        qs = [ "Q" if sol[x(i,j)] else "." for j in range(n) ]
        print(" ".join(qs))

#### 練習問題 5.7.2: nクイーンの自作プログラムとの比較



In [1]:
def queens0(n):
    def q(i, qs, qs1, qs2):
        if i == n:
            yield qs
        else:
            for j in range(0, n):
                if j not in qs and i+j not in qs1 and i-j not in qs2:
                    yield from q(i+1, qs+[j], qs1+[i+j], qs2+[i-j])
    yield from q(0, [], [], [])

In [1]:
import time
for n in [10,20,30]:
    t0 = time.time()
    q0 = next(queens0(n))
    t0 = time.time() - t0
    t1 = time.time()
    q1 = next(solutionsSAT(queens(n)))
    t1 = time.time() - t1
    print([n, t0, t1])

### 5.8 四色問題とグラフ彩色



グラフ彩色のためのCNF式



In [1]:
from cspsat import *
def graphColoring(vertices, edges, k, x=Bool("x")):
    for v in vertices:
        yield from eq1([ x(v,c) for c in range(1,k+1) ])
    for (u,v) in edges:
        for c in range(1,k+1):
            yield [ ~x(u,c), ~x(v,c) ]

例題「グラフ彩色」のグラフの定義



In [1]:
vertices = range(7)
edges = []
for v in vertices:
    if v == 0:
        edges.extend([ (v,u) for u in range(1,7) ])
    elif v <= 3:
        edges.extend([ (v,u) for u in range(v+1,7) if u != v+3 ])

例題「グラフ彩色」の解



In [1]:
solveSAT(graphColoring(vertices, edges, 4), positiveOnly=True)


# Table of Contents







#### 練習問題 5.8.1: n次マクレガーグラフの彩色



In [1]:
from cspsat.examples.graph import *
(vertices, edges) = mcgregorGraph(3)
solveSAT(graphColoring(vertices, edges, 4), positiveOnly=True)

#### 練習問題 5.8.2: クイーングラフ彩色



In [1]:
from itertools import combinations
def queenGraph(n):
    vertices = [ (i,j) for i in range(n) for j in range(n) ]
    edges = []
    for ((i,j),(k,l)) in combinations(vertices, 2):
            if i == k or j == l or i+j == k+l or i-j == k-l:
                    edges.append(((i,j), (k,l)))
    return (vertices, edges)

In [1]:
(vertices, edges) = queenGraph(5)
solveSAT(graphColoring(vertices, edges, 4), positiveOnly=True)
solveSAT(graphColoring(vertices, edges, 5), positiveOnly=True)
(vertices, edges) = queenGraph(6)
solveSAT(graphColoring(vertices, edges, 6), positiveOnly=True)
solveSAT(graphColoring(vertices, edges, 7), positiveOnly=True)

#### 練習問題 5.8.3: グラフの無線彩色



In [1]:
from cspsat import *
def radioColoring(vertices, edges, k, d=1, x=Bool("x")):
    for v in vertices:
        yield from eq1([ x(v,c) for c in range(1,k+1) ])
    for (u,v) in edges:
        for c1 in range(1,k+1):
            for c2 in range(1,k+1):
                if -d < c1-c2 < d:
                    yield [ ~x(u,c1), ~x(v,c2) ]

In [1]:
vertices = range(7)
edges = []
for v in vertices:
    if v == 0:
        edges.extend([ (v,u) for u in range(1,7) ])
    elif v <= 3:
        edges.extend([ (v,u) for u in range(v+1,7) if u != v+3 ])
solveSAT(radioColoring(vertices, edges, 7, d=2), positiveOnly=True)

### 5.9 論理回路



3ビット×2ビットの乗算回路のブール連鎖



In [1]:
from cspsat import *
def mul32bchain(xx, yy, zz):
    (a, b, c, t) = (Bool(), Bool(), Bool(), Bool())
    for i in range(3):
        yield ["equ", a(i), ["and", xx[i], yy[0]]]
    for i in range(3):
        yield ["equ", b(i), ["and", xx[i], yy[1]]]
    yield ["equ", zz[0], a(0)]
    yield ["equ", zz[1], ["xor", a(1), b(0)]]
    yield ["equ", c(0), ["and", a(1), b(0)]]
    yield ["equ", t(1), ["xor", a(2), b(1)]]
    yield ["equ", t(2), ["and", a(2), b(1)]]
    yield ["equ", zz[2], ["xor", t(1), c(0)]]
    yield ["equ", t(3), ["and", t(1), c(0)]]
    yield ["equ", c(1), ["or", t(2), t(3)]]
    yield ["equ", zz[3], ["xor", b(2), c(1)]]
    yield ["equ", c(2), ["and", b(2), c(1)]]
    yield ["equ", zz[4], c(2)]

21を3ビット×2ビットの積へ因数分解



In [1]:
(x, y) = (Bool("x"), Bool("y"))
xx = [ x(0), x(1), x(2) ]
yy = [ y(0), y(1) ]
zz = [ TRUE, FALSE, TRUE, FALSE, TRUE ]
bchain = mul32bchain(xx, yy, zz)
cnf = [ clause for c in bchain for clause in toCNF(c) ]
solveSAT(cnf, num=0)

Daddaアルゴリズムによるブール連鎖の生成



In [1]:
(x, y, z) = (Bool("x"), Bool("y"), Bool("z"))
xx = [ x(i) for i in range(3) ]
yy = [ y(i) for i in range(2) ]
zz = [ z(i) for i in range(5) ]
binary = Binary()
binary.addMul(xx, yy)
for c in binary.toBchain(zz):
    print(c)

18446743979220271189を32ビットの積に因数分解



In [1]:
def factor(n, m):
    (x, y, z) = (Bool("x"), Bool("y"), Bool("z"))
    xx = [ x(i) for i in range(m) ]
    yy = [ y(i) for i in range(m) ]
    zz = [ z(i) for i in range(2*m) ]
    binary = Binary()
    binary.addMul(xx, yy)
    cnf = [*binary.toCNF(zz), *Binary.eqK(zz, n)]
    for sol in solutionsSAT(cnf):
        xv = sum([ sol[x] << i for (i,x) in enumerate(xx) ])
        yv = sum([ sol[y] << i for (i,y) in enumerate(yy) ])
        zv = sum([ sol[z] << i for (i,z) in enumerate(zz) ])
        print([xv, yv, zv])

factor(18446743979220271189, 32)

#### 5.9.1 補足



In [1]:
(x, y) = (Var("x"), Var("y"))
csp = [
    ["int", x, 1, 2**32-1],
    ["int", y, 1, 2**32-1],
    ["mulCmp", "==", x, y, 18446743979220271189]
]
solveCSP(csp, encoder="log")


# Table of Contents





